In [3]:
from modified_diffusers import StableDiffusionControlNetPipeline, ControlNetModel
import torch

pretrained_path = 'lint/simpathizer'

pipe = StableDiffusionControlNetPipeline.from_pretrained(
    pretrained_path,
    controlnet=ControlNetModel.from_pretrained(pretrained_path, subfolder='controlnet'),
    safety_checker=None,
    feature_extractor=None,
    requires_safety_checker=False,
)

for component in pipe.components.values():
    if hasattr(component, 'device'):
        component.to('cuda', torch.float16)

pipe.enable_xformers_memory_efficient_attention()

Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

In [ ]:
import os

output_dir = "archive/mix_modes"
os.makedirs(output_dir, exist_ok=True)

for mode in [None, "+w", "-w", "+h", "-h", "+c", "-c"]:
    result = pipe(
        prompt="beautiful woman with blue eyes", 
        generator=torch.Generator().manual_seed(2048), 
        guidance_scale=7.5,
        controlnet_conditioning_scale=0.5,
        controlnet_prompt="1girl, blue eyes",
        controlnet_mode=mode # should be one of [None, "+w", "-w", "+h", "-h", "+c", "-c"]
    )
    result.images[0].save(f"{output_dir}/mode_{mode}.png")

In [ ]:
import numpy as np
conditioning_scales = np.linspace(0, 1, 11)

output_dir = "archive/woman_blue_eyes"
os.makedirs(output_dir, exist_ok=True)

for cond_scale in conditioning_scales:
    result = pipe(
        prompt="beautiful woman with blue eyes", 
        generator=torch.Generator().manual_seed(2048), 
        controlnet_conditioning_scale=np.around(cond_scale,1),
        controlnet_prompt="1girl, blue eyes",
        controlnet_mode=None # should be one of [None, "+w", "-w", "+h", "-h", "+c", "-c"]
    )

    result.images[0].save(f'{output_dir}/conditioning_scale_{np.around(cond_scale,1)}.png')

In [ ]:
from PIL import Image
from torchvision.utils import make_grid
from torchvision.io import read_image
from pathlib import Path

images = []
image_dir = './archive/style_mixing_modes'

for image_path in sorted(Path(image_dir).glob('*.png')):
    images.append(read_image(str(image_path)))

grid = make_grid(images, nrow=11)
grid = grid.permute(1,2,0).numpy()
image_grid = Image.fromarray(grid)
image_grid.save(Path(image_dir).parent/f'{Path(image_dir).name}_grid.png')